In [1]:
from ml_battery import *
import ml_battery.nhts_data as nhts_data
import sklearn.pipeline
import pickle
import pandas as pd
import numpy as np
import sklearn.ensemble
import sklearn.svm
import sklearn.linear_model
import sklearn.cross_validation

Using TensorFlow backend.
/rufous/occupancy_detection/modules/modules/build/all_of_it/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
USE_PRETRAINED=False

In [3]:
df = nhts_data.load_nhts.household()
data_train, data_test = sklearn.cross_validation.train_test_split(df, test_size=0.2, random_state=0)

In [4]:
X_columns = set(df.columns.values.tolist()) - set(["HHVEHCNT", "WTHHFIN", "HOUSEID"])
X_columns = sorted(X_columns)

y_columns = ["HHVEHCNT"]

weight_column = "WTHHFIN"

X_train, y_train, weights_train = data_train[X_columns], data_train[y_columns], data_train[weight_column]
X_test, y_test, weights_test = data_test[X_columns], data_test[y_columns], data_test[weight_column]
weights_train = np.ones(weights_train.shape)
weights_test = np.ones(weights_test.shape)

In [5]:
codebook = nhts_data.load_nhts.codebook()[['Name','Label','Type']].dropna().set_index('Name')
print(codebook.loc["BIKE"]["Label"], codebook.loc["BIKE"]["Type"])

Frequency of Bicycle Use for Travel C


In [6]:
clf = sklearn.pipeline.Pipeline([
    ('data_transformation', CodebookTransformer(codebook, X_possible_values = df)),
    ('feature_selection', SelectFromModelPandas(sklearn.ensemble.RandomForestClassifier())),
    ('classification', StackedClassifier(
        MultiClassifier([
            ("random forest", sklearn.ensemble.RandomForestClassifier(n_jobs=-1, max_depth=9)),
            ("logistic regression", sklearn.linear_model.LogisticRegression(C=100000)),
            ("one-hidden mlp, 1000 epochs, optimized hidden", OneLayerNNClassifier(n_epochs=1000, regularization=0.01, dropout=1, n_hidden=8)),
            #("linear SVC", sklearn.svm.LinearSVC(C=100000, max_iter=10000)),
            #("rbf SVC", sklearn.svm.SVC(kernel="rbf")),
            #("linear regression (rounded to nearest int)", IntegerRegressor(sklearn.linear_model.LinearRegression())),
            ("naive bayes", MixedNB()),
            #("one-hidden MLP, 1000 epochs, 20 hidden", OneLayerNNClassifier(20, 1000, batch_size=300, learning_rate=0.001)),
            #("one-hidden MLP, 1000 epochs, 1 hidden", OneLayerNNClassifier(1,1000)),
            #("one-hidden MLP, 10000 epochs, 1 hidden", OneLayerNNClassifier(1,10000)),
            ("stratified dummy", PatchedDummy()),
            ("ordered probit", OrdinalRegression("probit")),
            ("nested ordinal logit", NestedClassifier(sklearn.linear_model.LogisticRegression(C=100000), [0,[1,[2,[3,[4,[5,[6,[7,[8,[9,[10,[11,12]]]]]]]]]]]])),
        ], parallel=True),
        sklearn.linear_model.LogisticRegression(C=100000)
    ))
])

In [ ]:
if USE_PRETRAINED:
    with open("../models/NHTS_pipeline7.p","rb") as f:
        toy_pipeline = pickle.load(f)
    clf=toy_pipeline
else:
    print("fitting!")
    clf.fit(X_train, y_train, classification__sample_weight=weights_train)
    with open("../models/NHTS_pipeline7.p","wb") as f:
        pickle.dump(clf,f)

fitting!


/rufous/occupancy_detection/modules/modules/build/all_of_it/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/rufous/occupancy_detection/modules/modules/build/all_of_it/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/rufous/occupancy_detection/modules/modules/build/all_of_it/lib/python3.6/site-packages/sklearn/feature_selection/from_model.py:169: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)


In [20]:
(stacked_scores, stacked_cm, stacked_fi), (multi_scores, multi_cm, multi_fi) = clf.score(X_test, y_test, sample_weight=weights_test)

/rufous/occupancy_detection/modules/modules/build/all_of_it/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/rufous/occupancy_detection/modules/modules/build/all_of_it/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/rufous/occupancy_detection/modules/modules/build/all_of_it/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/rufous/occupancy_detection/modules/modules/build/all_of_it/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no pred

In [ ]:
multi_cm

In [ ]:
stacked_scores

In [ ]:
multi_scores

In [ ]:
multi_fi.sort_values(by="nested ordinal logit", ascending=False)

In [ ]:
pd.DataFrame(clf.steps[1][1].estimator_.feature_importances_, index=clf.steps[0][1].transform(X_train).columns).sort_values(by=0, ascending=False)

In [ ]:
multi_scores["stacked model"] = stacked_scores
print(multi_scores.round(3).to_latex())

In [ ]:
df.shape

In [ ]:
a = clf.steps[0][1].transform(df[X_columns])

In [ ]:
a.shape

In [ ]:
sklearn.pipeline.Pipeline(clf.steps[:2]).transform(df[X_columns]).shape

In [ ]:
clf.steps[2][1].multiestimator.estimators